<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
补充代码来自 <a href="http://mng.bz/orYv">从零构建大型语言模型</a> 一书，作者为 <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>代码仓库: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>

# 第2章：处理文本

本笔记本中使用的包如下：

In [16]:
from importlib.metadata import version

# 打印torch库的版本
print("torch 版本:", version("torch"))
# 打印tiktoken库的版本
print("tiktoken 版本:", version("tiktoken"))

torch 版本: 2.2.2
tiktoken 版本: 0.6.0


- 本章涵盖数据准备和采样，以使输入数据为大型语言模型（LLM）做好“准备”

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/01.webp?timestamp=1" width="500px">

## 2.1 理解词嵌入

- 本节无代码

- 嵌入形式有很多种；本书中我们专注于文本嵌入

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/02.webp" width="500px">

- 大型语言模型（LLM）处理的是高维空间中的嵌入（即数千维）
- 由于我们无法可视化如此高维的空间（人类的思维通常局限于1维、2维或3维），下图展示了一个二维的嵌入空间

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/03.webp" width="300px">

## 2.2 文本的分词处理

- 在本节中，我们将文本进行分词处理，也就是将文本拆分为更小的单元，比如单个单词和标点符号

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/04.webp" width="300px">

- 加载我们要处理的原始文本
- [《裁决》 作者：伊迪丝·华顿](https://en.wikisource.org/wiki/The_Verdict) 是一篇公共领域的短篇小说

In [17]:
import os
import urllib.request

# 检查文件是否已存在，如果不存在则下载文件
if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    # 从指定URL下载文件并保存为"the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

In [18]:
# 打开文件“the-verdict.txt”，读取其中的内容
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

# 打印文本的总字符数
print("字符总数:", len(raw_text))

# 打印前99个字符的内容
print(raw_text[:99])

字符总数: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


- 目标是将此文本进行分词处理并嵌入到大型语言模型中
- 让我们基于一些简单的示例文本开发一个简单的分词器，之后可以将其应用于上述文本
- 以下正则表达式将基于空白字符进行分割

In [6]:
import re

# 示例文本
text = "Hello, world. This, is a test."

# 使用正则表达式基于空白字符进行分割
result = re.split(r'(\s)', text)

# 打印分词后的结果
print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


- 我们不仅要基于空白字符进行分割，还需要对逗号和句号进行分割，因此我们将修改正则表达式以实现该功能

In [7]:
# 修改正则表达式以同时基于逗号、句号和空白字符进行分割
result = re.split(r'([,.]|\s)', text)

# 打印分割后的结果
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


- 正如我们所见，这会生成一些空字符串，让我们将它们移除

In [8]:
# 去除每个项中的空白字符，然后过滤掉任何空字符串
result = [item for item in result if item.strip()]

# 打印过滤后的结果
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


- 这看起来不错，但我们还需要处理其他类型的标点符号，例如句号、问号等

In [9]:
# 示例文本，包含多种标点符号
text = "Hello, world. Is this-- a test?"

# 正则表达式，处理逗号、句号、分号、问号、感叹号、括号、引号、双破折号和空白字符
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)

# 去除空白并过滤掉空字符串
result = [item.strip() for item in result if item.strip()]

# 打印结果
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


- 这已经非常不错了，现在我们可以将这个分词方法应用到原始文本中了

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/05.webp" width="350px">

In [10]:
# 对原始文本应用分词处理
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)

# 去除空白并过滤掉空字符串
preprocessed = [item.strip() for item in preprocessed if item.strip()]

# 打印前30个分词结果
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


- 让我们计算分词后的总词元数

In [11]:
# 打印分词后的总词元数
print(len(preprocessed))

4690


## 2.3 将词元转换为词元ID

- 接下来，我们将文本词元转换为词元ID，之后可以通过嵌入层进行处理

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/06.webp" width="500px">

- 从这些词元中，我们现在可以构建一个包含所有唯一词元的词汇表

In [12]:
# 对所有词元去重并排序，构建词汇表
all_words = sorted(set(preprocessed))

# 计算词汇表的大小
vocab_size = len(all_words)

# 打印词汇表大小
print(vocab_size)

1130


In [13]:
# 将词汇表中的每个词元映射为唯一的整数ID
vocab = {token: integer for integer, token in enumerate(all_words)}

- 下面是词汇表中的前50个条目：

In [14]:
# 打印词汇表中的前50个条目
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


- 下面，我们用一个小词汇表来演示对一个简短示例文本的分词处理：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/07.webp?123" width="500px">

- 现在将所有内容整合到一个分词器类中

In [15]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        # 初始化时，将词汇表映射为词元字符串到整数ID的映射
        self.str_to_int = vocab
        # 创建整数ID到词元字符串的反向映射
        self.int_to_str = {i: s for s, i in vocab.items()}
    
    def encode(self, text):
        # 使用正则表达式对输入文本进行分词
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        # 去除空白字符并过滤掉空字符串
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        # 将分词结果转换为词元ID
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        # 将词元ID转换为原始文本
        text = " ".join([self.int_to_str[i] for i in ids])
        # 在标点符号前去除空格
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

- `encode` 函数将文本转换为词元ID
- `decode` 函数将词元ID转换回文本

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/08.webp?123" width="500px">

- 我们可以使用分词器将文本编码（即分词）为整数
- 这些整数之后可以作为嵌入，作为大型语言模型（LLM）的输入

In [16]:
# 使用词汇表初始化分词器
tokenizer = SimpleTokenizerV1(vocab)

# 示例文本
text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""

# 将文本编码为词元ID
ids = tokenizer.encode(text)

# 打印词元ID
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


- 我们可以将这些整数解码回文本

In [17]:
# 将词元ID解码回文本
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [18]:
# 将文本编码为词元ID后，再将其解码回文本
tokenizer.decode(tokenizer.encode(text))

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

## 2.4 添加特殊的上下文词元

- 为未知词元和标记文本结束添加一些“特殊”词元是非常有用的

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/09.webp?123" width="500px">

- 一些分词器使用特殊词元来为大型语言模型（LLM）提供额外的上下文信息
- 其中一些特殊词元包括：
  - `[BOS]` （序列开始）用于标记文本的开始
  - `[EOS]` （序列结束）用于标记文本的结束（通常用于连接多个不相关的文本，例如两篇不同的维基百科文章或两本不同的书籍，等等）
  - `[PAD]` （填充）用于在训练LLM时批处理大小大于1的情况下，填充长度不同的文本，以使所有文本长度一致
  - `[UNK]` 表示词汇表中未包含的词

- 请注意，GPT-2 不需要上述的这些特殊词元，它只使用 `<|endoftext|>` 词元来简化处理
- `<|endoftext|>` 类似于上述的 `[EOS]` 词元
- GPT 也使用 `<|endoftext|>` 作为填充词元（因为我们通常在训练批量输入时使用掩码，不会关注填充的词元，所以这些词元具体是什么并不重要）
- GPT-2 不使用 `<UNK>` 词元来表示词汇表外的词汇；相反，GPT-2 使用字节对编码（BPE）分词器，它将词汇分解为子词单位，关于这一点我们将在后面的章节中讨论

- 我们在两个独立的文本来源之间使用 `<|endoftext|>` 词元：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/10.webp" width="500px">

- 让我们看看对以下文本进行分词会发生什么：

In [19]:
# 使用词汇表初始化分词器
tokenizer = SimpleTokenizerV1(vocab)

# 示例文本
text = "Hello, do you like tea. Is this-- a test?"

# 将文本编码为词元ID
tokenizer.encode(text)

KeyError: 'Hello'

- 上述代码产生了错误，因为单词 "Hello" 不在词汇表中
- 为了解决这种情况，我们可以在词汇表中添加像 `"<|unk|>"` 这样的特殊词元来表示未知单词
- 既然我们已经在扩展词汇表了，那就再添加一个名为 `"<|endoftext|>"` 的词元，这是在 GPT-2 训练中用于标记文本结束的词元（在连接的文本之间也使用，比如当我们的训练数据集中包含多篇文章、书籍等时）

In [20]:
# 对所有词元去重并排序，构建词汇表
all_tokens = sorted(list(set(preprocessed)))

# 扩展词汇表，添加特殊词元 "<|endoftext|>" 和 "<|unk|>"
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

# 将词汇表中的每个词元映射为唯一的整数ID
vocab = {token: integer for integer, token in enumerate(all_tokens)}

In [21]:
# 打印词汇表中的词元总数
len(vocab.items())

1132

In [22]:
# 打印词汇表中最后5个词元及其对应的ID
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


- 我们还需要相应地调整分词器，使其知道何时以及如何使用新的 `<unk>` 词元

In [23]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        # 初始化时，将词汇表映射为词元字符串到整数ID的映射
        self.str_to_int = vocab
        # 创建整数ID到词元字符串的反向映射
        self.int_to_str = {i: s for s, i in vocab.items()}
    
    def encode(self, text):
        # 使用正则表达式对输入文本进行分词
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        # 去除空白字符并过滤掉空字符串
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        # 如果词元不在词汇表中，使用 "<|unk|>" 作为未知词元
        preprocessed = [
            item if item in self.str_to_int 
            else "<|unk|>" for item in preprocessed
        ]
        # 将分词结果转换为词元ID
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        # 将词元ID转换为原始文本
        text = " ".join([self.int_to_str[i] for i in ids])
        # 在标点符号前去除空格
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

让我们使用修改后的分词器尝试对文本进行分词：

In [15]:
# 使用更新后的分词器
tokenizer = SimpleTokenizerV2(vocab)

# 两段示例文本
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

# 使用 "<|endoftext|>" 将两段文本连接起来
text = " <|endoftext|> ".join((text1, text2))

# 打印连接后的文本
print(text)

NameError: name 'SimpleTokenizerV2' is not defined

In [25]:
# 对连接后的文本进行编码（分词）
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [26]:
# 对文本进行编码并解码回原始文本
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

## 2.5 字节对编码 (BytePair Encoding)

- GPT-2 使用了字节对编码（BPE）作为其分词器
- 它允许模型将不在其预定义词汇表中的单词分解为更小的子词单元甚至是单个字符，从而能够处理词汇表外的单词
- 例如，如果 GPT-2 的词汇表中没有“unfamiliarword”这个词，它可能会将其分词为 ["unfam", "iliar", "word"]，或根据其训练的 BPE 合并规则进行其他子词拆分
- 原始的 BPE 分词器可以在这里找到：[https://github.com/openai/gpt-2/blob/master/src/encoder.py](https://github.com/openai/gpt-2/blob/master/src/encoder.py)
- 在本章中，我们使用来自 OpenAI 开源库 [tiktoken](https://github.com/openai/tiktoken) 的 BPE 分词器，它的核心算法使用 Rust 实现，以提高计算性能
- 我创建了一个笔记本，在 [./bytepair_encoder](../02_bonus_bytepair-encoder) 中对这两种实现进行了比较（在样本文本上，tiktoken 的速度快了约 5 倍）

In [25]:
# 使用以下命令安装 tiktoken 库
# pip install tiktoken

In [1]:
import importlib
import tiktoken

# 打印 tiktoken 库的版本
print("tiktoken 版本:", importlib.metadata.version("tiktoken"))

tiktoken 版本: 0.6.0


In [2]:
# 获取 GPT-2 的 BPE 分词器
tokenizer = tiktoken.get_encoding("gpt2")

In [4]:
# 示例文本
# text = (
#     "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
#     " of someunknownPlace."
# )
text = (
    "[0.2]"
)
# 使用 tiktoken 分词器对文本进行编码，允许使用特殊词元 "<|endoftext|>"
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

# 打印编码后的整数ID
print(integers)

[58, 15, 13, 17, 60]


In [5]:
# 使用 tiktoken 分词器对整数ID进行解码
strings = tokenizer.decode(integers)

# 打印解码后的文本
print(strings)

[0.2]


- BPE 分词器将未知单词拆分为子词和单个字符：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/11.webp" width="300px">

## 2.6 使用滑动窗口进行数据采样

- 我们训练大型语言模型时，每次生成一个词，因此我们希望相应地准备训练数据，其中序列中的下一个词是要预测的目标：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/12.webp" width="400px">

In [33]:
# 打开文件“the-verdict.txt”，读取其中的内容
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

# 使用 tiktoken 分词器对文本进行编码
enc_text = tokenizer.encode(raw_text)

# 打印编码后的词元总数
print(len(enc_text))

5145


- 对于每个文本块，我们需要准备输入和目标
- 由于我们希望模型预测下一个词，因此目标是将输入向右移动一个位置

In [34]:
# 从编码后的文本中获取一个样本，从第50个词元开始
enc_sample = enc_text[50:]

In [37]:
# 设置上下文大小为4
context_size = 4

# 取出输入 x 为样本的前4个词元
x = enc_sample[:context_size]

# 取出目标 y 为样本向右偏移一个位置的4个词元
y = enc_sample[1:context_size+1]

# 打印输入 x 和目标 y
print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


- 逐步预测的过程如下所示：

In [38]:
# 循环遍历，逐步打印每一步的输入上下文和对应的目标词元
for i in range(1, context_size + 1):
    context = enc_sample[:i]   # 输入上下文，长度从1到i
    desired = enc_sample[i]    # 对应的目标词元

    # 打印上下文及其目标
    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [39]:
# 循环遍历，逐步打印每一步的解码后的输入上下文和对应的目标词元
for i in range(1, context_size + 1):
    context = enc_sample[:i]    # 输入上下文，长度从1到i
    desired = enc_sample[i]     # 对应的目标词元

    # 打印解码后的上下文及其目标
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


- 我们将在后面的章节中处理下一个词的预测，在此之前我们会讲解注意力机制
- 现在，我们实现一个简单的数据加载器，它遍历输入数据集，并返回向右偏移一个位置的输入和目标

- 安装并导入 PyTorch（安装提示请参见附录A）

In [41]:
import torch
# 打印 PyTorch 的版本
print("PyTorch 版本:", torch.__version__)

PyTorch 版本: 2.2.2


- 我们使用滑动窗口方法，位置移动 +1：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/13.webp?123" width="500px">

- 创建数据集和数据加载器，从输入文本数据集中提取块

In [42]:
from torch.utils.data import Dataset, DataLoader

# 定义 GPT 数据集类
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # 对整个文本进行分词
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # 使用滑动窗口将文本分割为最大长度为 max_length 的重叠序列
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    # 返回数据集的长度
    def __len__(self):
        return len(self.input_ids)

    # 根据索引返回对应的输入和目标序列
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [43]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # 初始化分词器
    tokenizer = tiktoken.get_encoding("gpt2")

    # 创建数据集
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # 创建数据加载器
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

- 让我们使用批量大小为1，且上下文大小为4的设置来测试数据加载器：

In [44]:
# 打开文件“the-verdict.txt”，读取其中的内容
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [45]:
# 使用批量大小为1，上下文大小为4的设置创建数据加载器
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

# 获取数据加载器的迭代器
data_iter = iter(dataloader)

# 获取第一个批次数据
first_batch = next(data_iter)

# 打印第一个批次的数据
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [46]:
# 获取第二个批次数据
second_batch = next(data_iter)

# 打印第二个批次的数据
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


- 如下所示，使用与上下文长度相等的滑动步长（此处为4）的示例：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/14.webp" width="500px">

- 我们还可以创建批处理输出
- 请注意，这里我们增加了滑动步长，以避免批次之间的重叠，因为过多的重叠可能会导致过拟合

In [47]:
# 使用批量大小为8，上下文大小为4，滑动步长为4的设置创建数据加载器
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

# 获取数据加载器的迭代器
data_iter = iter(dataloader)

# 获取第一个批次的输入和目标
inputs, targets = next(data_iter)

# 打印输入和目标
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


## 2.7 创建词元嵌入

- 数据已经几乎准备好用于大型语言模型（LLM）
- 但最后，让我们使用嵌入层将词元嵌入到连续的向量表示中
- 通常，这些嵌入层是大型语言模型的一部分，并且在模型训练期间进行更新（训练）

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/15.webp" width="400px">

- 假设我们有以下四个输入示例，分词后的输入ID为2、3、5和1：

In [53]:
# 分词后的输入ID示例
input_ids = torch.tensor([2, 3, 5, 1])

- 为了简单起见，假设我们只有6个词的小词汇表，并且我们希望创建大小为3的嵌入向量：

In [54]:
# 定义词汇表大小为6，嵌入维度为3
vocab_size = 6
output_dim = 3

# 设置随机种子以确保结果可复现
torch.manual_seed(123)

# 创建嵌入层，词汇表大小为6，嵌入维度为3
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

- 这将生成一个 6x3 的权重矩阵：

In [55]:
# 打印嵌入层的权重矩阵（6x3）
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


- 对于熟悉独热编码的人来说，上述嵌入层的方法本质上是一种更高效的实现方式，相当于在全连接层中进行独热编码后进行矩阵乘法的操作，这在补充代码 [./embedding_vs_matmul](../03_bonus_embedding-vs-matmul) 中有描述
- 由于嵌入层只是独热编码和矩阵乘法方法的更高效实现，因此它可以被视为一个可以通过反向传播优化的神经网络层

- 要将ID为3的词元转换为3维向量，我们执行以下操作：

In [56]:
# 将ID为3的词元转换为3维嵌入向量
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


- 请注意，上述结果是 `embedding_layer` 权重矩阵的第4行
- 要嵌入之前定义的所有 `input_ids` 值，我们执行以下操作：

In [57]:
# 对所有输入ID进行嵌入操作
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


- 嵌入层本质上是一种查找操作：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/16.webp?123" width="500px">

- **你可能对比较嵌入层与常规线性层的附加内容感兴趣：[../03_bonus_embedding-vs-matmul](../03_bonus_embedding-vs-matmul)**

## 2.8 编码词的位置

- 嵌入层将ID转换为相同的向量表示，而不考虑它们在输入序列中的位置：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/17.webp" width="400px">

- 位置嵌入与词元嵌入向量相结合，形成大型语言模型的输入嵌入：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/18.webp" width="500px">

- BytePair 编码器的词汇表大小为 50,257：
- 假设我们希望将输入词元编码为 256 维的向量表示：

In [58]:
# 定义词汇表大小为50257，嵌入维度为256
vocab_size = 50257
output_dim = 256

# 创建词元嵌入层，词汇表大小为50257，嵌入维度为256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

- 如果我们从数据加载器中采样数据，我们会将每个批次中的词元嵌入为 256 维向量
- 如果批量大小为 8，每个批次包含 4 个词元，这将生成一个 8 x 4 x 256 的张量：

In [59]:
# 设置上下文最大长度为4
max_length = 4

# 使用批量大小为8，最大长度为4，步长为4的设置创建数据加载器
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)

# 获取数据加载器的迭代器
data_iter = iter(dataloader)

# 获取输入和目标数据
inputs, targets = next(data_iter)

In [62]:
# 打印词元ID
print("Token IDs:\n", inputs)

# 打印输入数据的形状
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [63]:
# 将输入词元通过嵌入层映射为嵌入向量
token_embeddings = token_embedding_layer(inputs)

# 打印嵌入向量的形状
print(token_embeddings.shape)

torch.Size([8, 4, 256])


- GPT-2 使用绝对位置嵌入，因此我们只需创建另一个嵌入层：

In [72]:
 # 上下文长度设置为最大长度
context_length = max_length 
# 创建位置嵌入层，输入维度为上下文长度，输出维度为输出维度
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)  

In [73]:
# 通过位置嵌入层获取位置嵌入，输入为从0到max_length的索引
pos_embeddings = pos_embedding_layer(torch.arange(max_length)) 

# 打印位置嵌入的形状
print(pos_embeddings.shape)  

torch.Size([4, 256])


- 要创建在 LLM 中使用的输入嵌入，我们只需将词元嵌入和位置嵌入相加：

In [74]:
# 将词元嵌入和位置嵌入相加，生成输入嵌入
input_embeddings = token_embeddings + pos_embeddings  

# 打印输入嵌入的形状
print(input_embeddings.shape)  

torch.Size([8, 4, 256])


- 在输入处理工作流的初始阶段，输入文本被分割成单独的词元
- 在此分割之后，这些词元根据预定义的词汇表被转换为词元ID：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/19.webp" width="400px">

# Summary and takeaways

查看 [./dataloader.ipynb](./dataloader.ipynb) 代码笔记本，这是我们在本章中实现的数据加载器的简洁版本，接下来训练 GPT 模型时将需要它。

请查看 [./exercise-solutions.ipynb](./exercise-solutions.ipynb) 获取练习的解决方案。